In [1]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [2]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Read in helper files

# Enables State Full Name to Abbreviation
state_dict = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/List-of-US-States/master/states.csv', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

# Filter to PCE top Level Categores
pce_levels = pd.read_csv('./supplemental_data/pce_categories.csv', header=None)
pce_levels = pce_levels[0].to_list()

Files cleaned in order of perceived importance

# Financial Data

### GDP by State and Year

In [4]:
df = pd.read_csv('./dirty/gdp_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [5]:
df.head(2)

,GeoFips,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,United States,3,Current-dollar GDP (millions of current dollars),8577552.00,9062817.00,9631172.00,10250952.00,10581929.00,10929108.00,11456450.00,12217196.00,13039197.00,13815583.00,14474228.00,14769862.00,14478067.00,15048970.00,15599731.00,16253970.00,16843196.00,17550687.00,18206023.00,18695106.00,19479623.00,20527159.00,21372582.00,20893746.00
1,0,United States,4,Compensation (millions of dollars),4713220.00,5075701.00,5409937.00,5854634.00,6046346.00,6143370.00,6362298.00,6729306.00,7077722.00,7491260.00,7889371.00,8068682.00,7767191.00,7932970.00,8234017.00,8575362.00,8843637.00,9259654.00,9709535.00,9977096.00,10436745.00,10969807.00,11459449.00,11580088.00


In [6]:
# reshape
df = pd.melt(df, id_vars=['GeoFips','GeoName','LineCode','Description'])

In [7]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,variable,value
2560,35000,New Mexico,3,Current-dollar GDP (millions of current dollars),2005,73911.40
2673,93000,Great Lakes,7,Taxes on production and imports (TOPI) (mill...,2005,139663.10
5413,2000,Alaska,7,Taxes on production and imports (TOPI) (mill...,2015,2974.40
2712,2000,Alaska,5,Gross operating surplus (millions of dollars),2006,22400.60
3458,34000,New Jersey,7,Taxes on production and imports (TOPI) (mill...,2008,42535.50


In [8]:
# rename
df = df.rename(columns={'variable':'year', 'value':'dollars'}) 

# Map full state names to Abbreviated form
df['state'] = df['GeoName'].map(state_dict)

In [9]:
# drop uneeded columns
df = df.drop(columns=['GeoFips','GeoName','LineCode'])

# Turn GDP Values into 1:1 dollars -- Multiply by 1 million
df['dollars'] = df['dollars']*1000000

# clean up column names a little bit
df['Description'] = df['Description'].str.replace("(millions of dollars)", "", regex=False)
df['Description'] = df['Description'].str.replace("(millions of current dollars)", "", regex=False)

In [10]:
df.sample(5)

,Description,year,dollars,state
3034,Subsidies,2007,-749800000.00,CO
2913,Taxes on production and imports (TOPI),2006,2400100000.00,SD
4712,Gross operating surplus,2012,22232100000.00,SD
1011,Compensation,2000,178341800000.00,MA
4008,Taxes on production and imports (TOPI),2010,19204800000.00,MD


In [11]:
df = df.pivot_table(index=['state','year'], # columns to keep
                    columns='Description', # unstack this column
                    values='dollars', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

Description,state,year,Subsidies,Taxes on production and imports (TOPI),Compensation,Current-dollar GDP,Gross operating surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [12]:
# Strip WhiteSpace
df.columns = df.columns.str.strip()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   state                                   1224 non-null   object 
 1   year                                    1224 non-null   object 
 2   Subsidies                               1224 non-null   float64
 3   Taxes on production and imports (TOPI)  1224 non-null   float64
 4   Compensation                            1224 non-null   float64
 5   Current-dollar GDP                      1224 non-null   float64
 6   Gross operating surplus                 1224 non-null   float64
dtypes: float64(5), object(2)
memory usage: 67.1+ KB


In [14]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [15]:
df.head(5)

Description,state,year,subsidies,taxes_on_production_and_imports_(topi),compensation,current-dollar_gdp,gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [16]:
# including `index=False` removes unwanted description level
df.to_csv('./clean/gdp_state_annual_clean.csv', index=False)

### PCE Total by Function -  State Annual

Will divide by population to get Per Capita PCE

In [17]:
df = pd.read_csv('./dirty/pce_function_state_annual.csv', skipfooter=4, engine='python')
# verified correct head/tail skip against manual excel inspections

In [18]:
df.head(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""56000""",Wyoming,7,SAPCE4,1,...,Personal consumption expenditures,Millions of current dollars,9363.50,9993.40,10698.30,11458.70,11914.40,12420.80,13215.80,14256.60,15388.00,16747.80,18164.60,18995.80,18550.80,19198.60,20177.90,21147.60,21807.30,22553.50,23022.70,23082.90,23564.20,24351.90,24839.00,24466.90
1,"""56000""",Wyoming,7,SAPCE4,2,...,Household consumption expenditures,Millions of current dollars,9261.70,9831.40,10510.30,11254.60,11687.40,12168.80,12880.80,13933.60,15129.00,16472.80,17876.60,18654.90,18181.70,18830.00,19781.20,20716.50,21349.40,22084.60,22483.20,22499.90,22959.10,23749.70,24252.90,23731.20


In [19]:
# drop initially unnecessary columns
df = df.drop(columns=['Region','TableName','LineCode','IndustryClassification'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoFIPS','GeoName','Description','Unit'])

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

# rename columns
df = df.rename(columns={'variable':'year', 'Description':'pce_description','value':'pce_value'})

# Convert to 1:1 dollars
df['pce_value'] = df['pce_value']*1000000

# final drop of unnecessary columns
df = df.drop(columns=['GeoName','Unit','GeoFIPS'])

In [20]:
df.head(5)

,pce_description,year,pce_value,state
0,Personal consumption expenditures,1997,9363500000.00,WY
1,Household consumption expenditures,1997,9261700000.00,WY
2,Food and beverages purchased for off-premise...,1997,919600000.00,WY
3,Food and nonalcoholic beverages purchased f...,1997,765300000.00,WY
4,Alcoholic beverages purchased for off-premi...,1997,151400000.00,WY


In [21]:
df['pce_description'].value_counts()

Personal consumption expenditures                                              24
    Admissions to specified spectator amusements                               24
   Commercial and vocational schools                                           24
   Nursery, elementary, and secondary schools                                  24
   Higher education                                                            24
                                                                               ..
     Pharmaceutical products                                                   24
    Pharmaceutical and other medical products                                  24
   Medical products, appliances, and equipment                                 24
  Health                                                                       24
  Less: Receipts from sales of goods and services by nonprofit institutions    24
Name: pce_description, Length: 133, dtype: int64

In [22]:
df['pce_description'] = df['pce_description'].str.strip()

df = df[df['pce_description'].isin(pce_levels)]

In [23]:
df.head(5)

,pce_description,year,pce_value,state
0,Personal consumption expenditures,1997,9363500000.00,WY
2,Food and beverages purchased for off-premises ...,1997,919600000.00,WY
6,"Clothing, footwear, and related services",1997,545900000.00,WY
17,"Housing, utilities, and fuels",1997,1780500000.00,WY
29,"Furnishings, household equipment, and routine ...",1997,419600000.00,WY


In [24]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='pce_description', # unstack this column
                    values='pce_value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

In [25]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [26]:
df.head(2)

pce_description,year,state,"clothing,_footwear,_and_related_services",communication,education,financial_services_and_insurance,food_and_beverages_purchased_for_off-premises_consumption,food_services_and_accommodations,"furnishings,_household_equipment,_and_routine_household_maintenance",health,"housing,_utilities,_and_fuels",net_foreign_travel_and_expenditures_abroad_by_u.s._residents,other_goods_and_services,personal_consumption_expenditures,recreation,transportation
0,1997,WY,545900000.00,185200000.00,178000000.00,789100000.00,919600000.00,684500000.00,419600000.00,1340100000.00,1780500000.00,-127800000.00,597700000.00,9363500000.00,794700000.00,1154700000.00
1,1998,WY,572500000.00,187800000.00,190500000.00,903500000.00,935300000.00,705500000.00,450200000.00,1441400000.00,1858500000.00,-85700000.00,615000000.00,9993400000.00,832900000.00,1224000000.00


In [27]:
# add PCE to value columns
cols = df.columns[~df.columns.str.contains('year|state')]

df.rename(columns = dict(zip(cols, 'pce_' + cols)), inplace=True)

In [28]:
df.head(2)

pce_description,year,state,"pce_clothing,_footwear,_and_related_services",pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,"pce_furnishings,_household_equipment,_and_routine_household_maintenance",pce_health,"pce_housing,_utilities,_and_fuels",pce_net_foreign_travel_and_expenditures_abroad_by_u.s._residents,pce_other_goods_and_services,pce_personal_consumption_expenditures,pce_recreation,pce_transportation
0,1997,WY,545900000.00,185200000.00,178000000.00,789100000.00,919600000.00,684500000.00,419600000.00,1340100000.00,1780500000.00,-127800000.00,597700000.00,9363500000.00,794700000.00,1154700000.00
1,1998,WY,572500000.00,187800000.00,190500000.00,903500000.00,935300000.00,705500000.00,450200000.00,1441400000.00,1858500000.00,-85700000.00,615000000.00,9993400000.00,832900000.00,1224000000.00


In [29]:
df.to_csv('./clean/pce_state_annual_clean.csv', index=False)

### Per Capita Personal Income

In [30]:
df = pd.read_csv('./dirty/personal_income_per_capita_state_annual.csv',skiprows=5, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [31]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
41,16000,Idaho,3,Per capita personal income (dollars) 2/,499,488,366,268,228,389,394,466,421,421,431,459,594,919,1040,1110,1144,1203,1273,1352,1297,1352.00,1532.00,1693.00,1596.00,1612.00,1651.00,1792.00,1857.00,1882.00,1959.00,1961.00,2035.00,2150.00,2230.00,2316.00,2615.00,2638.00,2808.00,2971.00,3382.00,3691.00,3918.00,4329.00,4885.00,5566.00,5850.00,6343.00,6726.00,7497.00,8009.00,8856.00,9600.00,9961.00,10795.00,11399.00,12004.00,12297.00,12918.00,13966.00,15232.00,16152.00,16509.00,17640.00,18688.00,19368.00,20252.00,21058.00,21506.00,22783.00,23688.00,25183.00,25799.00,26233.00,26728.00,28317.00,29392.00,31476.00,32707.00,32841.00,31281.00,32097.00,33579.00,35110.00,36095.00,37680.00,39386.00,40098.00,41560.00,43356.00,45261.00,48208.00,51379.00
154,56000,Wyoming,2,Population (persons) 1/,223000,226000,229000,230000,230000,233000,237000,240000,243000,246000,248000,250000,249000,249000,246000,240000,240000,253000,256000,269000,277000,290000.00,291000.00,293000.00,290000.00,293000.00,307000.00,312000.00,314000.00,315000.00,320000.00,331000.00,337000.00,333000.00,336000.00,339000.00,332000.00,323000.00,322000.00,324000.00,329000.00,333795.00,340023.00,346863.00,353353.00,364548.00,380477.00,395446.00,411530.00,430897.00,451850.00,474185.00,491712.00,506400.00,510344.00,504895.00,499696.00,495633.00,476966.00,465103.00,458373.00,453690.00,459260.00,466251.00,473081.00,480283.00,485160.00,488167.00,489451.00,490787.00,491780.00,494300.00,494657.00,500017.00,503453.00,509106.00,514157.00,522667.00,534876.00,546043.00,559851.00,564405.00,566858.00,575497.00,580930.00,580949.00,583645.00,581983.00,576248.00,574801.00,575341.00,577267.00,578803.00
21,9000,Connecticut,1,Personal income (millions of dollars),1640.5,1494.7,1313.1,1026.9,970.3,1091.2,1191,1362.2,1459.3,1312,1436.6,1589.7,2029.9,2588.3,2912.4,2921.7,2837.5,3058,3387.8,3508.9,3417.3,3867.60,4444.20,4860.80,5269.90,5373.80,5787.90,6323.50,6744.30,6538.00,6986.90,7276.10,7701.70,8227.20,8691.10,9334.80,10038.60,11023.50,12086.40,12828.80,14692.50,15673.80,16514.40,17804.70,19393.40,21157.20,22725.50,24536.40,27032.70,30080.90,33813.60,38493.90,43243.00,46701.30,49958.20,55880.70,60000.00,64553.30,70443.30,77668.20,84249.00,86889.10,87815.80,94269.30,97347.60,100108.30,105642.50,110746.30,117991.80,127302.10,134439.00,146946.40,155003.00,154596.40,156784.60,165149.40,174430.00,188872.60,203467.20,216174.50,212582.60,221058.50,226907.10,231171.10,226175.80,236756.60,243164.60,247777.40,253120.70,264263.10,270282.80,279612.40,295955.60
163,93000,Great Lakes,2,Population (persons) 1/,25187000,25332000,25426000,25535000,25632000,25694000,25824000,25960000,26096000,26243000,26456000,26725000,27042000,27158000,26478000,26511000,26379000,28425000,29153000,29832000,30324000,30530000.00,30883000.00,31498000.00,32166000.00,33063000.00,33779000.00,34404000.00,34967000.00,35578000.00,35928000.00,36290000.00,36616000.00,36927000.00,37357000.00,37868000.00,38405000.00,38951000.00,39347000.00,39645000.00,39904000.00,40319722.00,40622322.00,40824274.00,40946800.00,41036791.00,41105385.00,41186726.00,41353165.00,41510152.00,41610978.00,41693671.00,41647869.00,41492474.00,41366046.00,41393218.00,41417995.00,41455030.00,41589687.00,41720866.00,41872543.00,42091157.00,42495903.00,42902625.00,43274741.00,43589980.00,43923694.00,44239468.00,44493603.00,44727877.00,44969225.00,45216019.00,45401564.00,45550284.00,45707787.00,45844372.00,45949142.00,46071573.00,46187992.00,46275120.00,46355854.00,46450

In [32]:
# drop un-needed columns
df = df.drop(columns=['LineCode','GeoFips'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoName','Description'])

# rename columns
df = df.rename(columns={'variable':'year'})

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

In [33]:
df.sample(2)

,GeoName,Description,year,value,state
4344,Delaware,Personal income (millions of dollars),1953,825.00,DE
11134,Wyoming,Population (persons) 1/,1990,453690.00,WY


In [34]:
df.Description.value_counts()

Personal income (millions of dollars)      5580
Population (persons) 1/                    5580
Per capita personal income (dollars) 2/    5580
Name: Description, dtype: int64

In [35]:
# filter to only Per Capita Personal Income
df = df.loc[(df['Description']=='Per capita personal income (dollars) 2/') | (df['Description']=='Population (persons) 1/')]

In [36]:
# Clean up categorical values
df['Description'] = df['Description'].str.replace(' (persons) 1/','', regex=False)
df['Description'] = df['Description'].str.replace(' (dollars) 2/','', regex=False)

In [37]:
# remove nulls generated by region drop from state mapping
df = df.dropna()

df = df.drop(columns='GeoName')

In [38]:
df.head(5)

,Description,year,value,state
4,Population,1929,2644000,AL
5,Per capita personal income,1929,319,AL
10,Population,1929,430000,AZ
11,Per capita personal income,1929,598,AZ
13,Population,1929,1852000,AR


In [39]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='Description', # unstack this column
                    values='value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

In [40]:
df.to_csv('./clean/per_capita_personal_income-population_state_annual_clean.csv', index=False)

### Business Applications State Annual

In [41]:
df = pd.read_csv('./dirty/business_apps_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

# Data exists from 2005-2021

In [42]:
# NAIC Sector Codes
naic_sector_codes = pd.read_csv('./supplemental_data/naic_sectors_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

# NAIC Series Codes
naic_series_codes = pd.read_csv('./supplemental_data/naic_series_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

In [43]:
# Filter out incomplete years
df = df[df['year']!=2022]

df = df[df['year']!=2004]

# filter out unwanted series
df = df[~df['series'].isin(['BF_PBF4Q','BF_PBF8Q','BF_SBF4Q','BF_SBF8Q'])]

# filter to only seasonally adjusted data
df = df[df['sa']=='A']

In [44]:
df.tail(5)

,sa,naics_sector,series,geo,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
27275,A,TOTAL,BF_BF8Q,VA,2005,1070,1090,1043,1054,1036,1088,1009,1021,1044,1130,1103,1084
27277,A,TOTAL,BF_BF8Q,WA,2005,1161,1063,1078,1060,993,1109,1162,1156,840,1230,1197,1159
27279,A,TOTAL,BF_BF8Q,WV,2005,D,161,D,148,160,187,167,D,D,201,202,172
27281,A,TOTAL,BF_BF8Q,WI,2005,525,620,609,645,606,624,622,598,467,620,593,553
27283,A,TOTAL,BF_BF8Q,WY,2005,D,D,121,120,119,125,D,D,D,D,D,D


In [45]:
# convert selected columns by index(months) to numeric
df.iloc[:,5:17] = df.iloc[:,5:17].apply(pd.to_numeric, errors='coerce')

# create annual total
df['annual_total'] =  df.iloc[:,5:17].sum(axis=1, min_count=12)

# drop monthly columns after creating annual total
df = df.drop(df.columns[5:17], axis=1)

In [46]:
df.sample(5)

,sa,naics_sector,series,geo,year,annual_total
16767,A,TOTAL,BA_HBA,OK,2011,12450.00
21605,A,TOTAL,BA_CBA,MN,2008,9151.00
5443,A,TOTAL,BA_WBA,OK,2018,6145.00
18354,A,NAICS71,BA_HBA,US,2010,23876.00
13391,A,TOTAL,BA_HBA,MD,2013,22051.00


In [47]:
# Map supplemental NAIC Data to columns
df['naics_sector'] = df['naics_sector'].map(naic_sector_codes)

df['series'] = df['series'].map(naic_series_codes)

In [48]:
df = df.pivot_table(index=['sa','geo','year','naics_sector'], # columns to keep
                    columns='series', # unstack this column
                    values='annual_total', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

series,sa,geo,year,naics_sector,Business Applications,Business Applications from Corporations,Business Formations within Eight Quarters,Business Formations within Four Quarters,High‐Propensity Business Applications BA_WBA – Business Applications with Planned Wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [49]:
df.head(5)

series,sa,geo,year,naics_sector,Business Applications,Business Applications from Corporations,Business Formations within Eight Quarters,Business Formations within Four Quarters,High‐Propensity Business Applications BA_WBA – Business Applications with Planned Wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [50]:
df.to_csv('./clean/business_apps_state_annual_clean.csv', index=False)

### Poverty Rate

In [51]:
df = pd.read_csv('./dirty/poverty_rate_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [52]:
df.sample(2)

,STATE,Percent,Year,Unnamed: 3
1165,Texas,15.10,1998,NaN
137,Ohio,11.90,2018,NaN


In [53]:
# drop unused columns
df = df.drop(columns='Unnamed: 3')

# convert to numeric percent
df['Percent'] = (df['Percent']/100)

# Map states to abbreviated form
df['STATE'] = df['STATE'].map(state_dict)

# convert column names to lower case
df.columns = df.columns.str.lower()

df = df.rename(columns={'percent':'poverty_rate'})

In [54]:
df.sample(2)

,state,poverty_rate,year
374,KY,0.22,2013
452,UT,0.11,2012


In [55]:
df.to_csv('./clean/poverty_rate_state_annual_clean.csv', index=False)

# Education Data

### Total Spending per Student

In [56]:
df = pd.read_csv('./dirty/tot_spend_student_state_annual.csv', skiprows=6, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [57]:
# Create String
string = 'Total Current Expenditures for Public El-Sec (TE5) per Pupil (MEMBR) [State Finance]'

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

In [58]:
df.sample(1)

,State Name,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,2012-13,2011-12,2010-11,2009-10,2008-09,2007-08,2006-07,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98
24,MISSISSIPPI,9253,8909,8755,8692,8445,8265,8117,8097,7926,8104,8064,7890,7459,7173,6548,6199,5792,5354,5175,5014,4565,4288


In [59]:
df['State Name'].unique().shape

(51,)

In [60]:
# reshape dataframe
df = pd.melt(df, id_vars=['State Name'])

#rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_expenditure'})

# trim to desired year digits
df['start_year'] = df['start_year'].str[:5]

# convert state dictionary to uppercase values - match dataframe format
state_dict_upper = {k.upper():v.upper() for k,v in state_dict.items()}

# map state name to state abbreviation
df['state'] = df['state'].map(state_dict_upper)

In [61]:
# Check output
df.head(5)

,state,start_year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [62]:
df.state.unique().shape

(51,)

In [63]:
df.to_csv('./clean/tot_spend_student_state_annual_clean.csv', index=False)

### Total Revenue Per Student

In [64]:
df = pd.read_csv('./dirty/tot_rev_per_student_state_annual.csv', skiprows=0, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [65]:
df.head(5)

,State Name,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2018-19,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2017-18,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2016-17,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2015-16,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2014-15,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2013-14,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2012-13,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2011-12,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2010-11,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2009-10,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2008-09,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2007-08,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2006-07,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2005-06,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2004-05,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2003-04,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2002-03,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2001-02,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2000-01,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1999-00,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1998-99,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1997-98
0,ALABAMA,11415,10792,10590,10200,9992,9913,9653,9534,9776,9667,9708,10356,9548,8555,8028,7349,6971,6956,6503,6523,5975,5535
1,ALASKA,19553,19038,18897,18851,22379,20447,20312,19034,18699,17759,17319,17471,14304,12849,12632,11576,10928,10717,10275,10118,9532,9222
2,ARIZONA,10396,9697,9214,9293,8995,8703,8616,8613,9111,9343,9002,9457,9023,8071,7814,7550,7791,7214,7071,6455,5988,5812
3,ARKANSAS,11733,11592,11388,10939,10762,10478,10391,10939,10939,10738,10072,9758,9362,9031,8712,7542,7243,7112,6250,6054,5772,5697
4,CALIFORNIA,16014,14535,13965,13606,11786,10985,10481,10590,10790,10239,11180,11228,10857,9909,9234,8980,8975,8363,8306,7465,6750,6572


In [66]:
# Create String
string = 'Total Revenues (TR) per Pupil (MEMBR) [State Finance] '

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

# Reshape df
df = pd.melt(df, id_vars=['State Name'])

# Rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_revenue'})

# remap state name to state abbreviations
df['state'] = df['state'].map(state_dict_upper)

# Use Start year instead of year-span
df['start_year'] = df['start_year'].str[:4]

In [67]:
# Sample output
df.sample(10)

,state,start_year,per_pupil_revenue
1056,OK,1998,5811
226,MI,2014,12649
246,TN,2014,9471
1022,AZ,1998,5988
228,MS,2014,9269
468,FL,2009,9891
479,MD,2009,15738
898,NM,2001,8161
783,LA,2003,7951
251,WA,2014,12673


In [68]:
df.to_csv('./clean/tot_revenue_student_state_annual_clean.csv', index=False)